In [ ]:
#file_negatif
#file_positif

In [ ]:
data = {}
for neg in negative:
    data[neg] = "negatif"
for pos in positive:
    data[pos] = "positif"

In [ ]:
def set_kata_unik(data):
    kalimat = []
    sentimen = []
    kata_unik = []
    for kal, sen in data.items():
        kalimat.append(kal)
        sentimen.append(sen)
        for ts in kal.strip().split():
            kata_unik.append(ts)

    kata_unik = set(kata_unik)
    
    return kata_unik

In [ ]:
def set_vector(kata_unik):
    list_vec = []
    for tx in txt:
        kata_unik_ = dict.fromkeys(kata_unik, 0)
        for t in tx.strip().replace("   ", " ").replace("  ", " ").split(" "):
            try:
                kata_unik_[t] += 1
            except Exception as e:
                print(e)
                print(tx)
        list_vec.append(kata_unik_)
        
    
    return list_vec

In [ ]:
list_vec = set_vector(data)

In [ ]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
        
    return tfDict

In [ ]:
tf = []
for i in range(len(list_vec)):
    tf.append(computeTF(list_vec[i], txt[i].strip().replace("   ", " ").replace("  ", " ").split(" ")))

In [ ]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [ ]:
idfs = computeIDF(tf)

In [ ]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [ ]:
final_vec = []
for i in range(len(tf)):
    tfidf = computeTFIDF(tf[i], idfs)
    value = []
    for k, v in tfidf.items():
        value.append(v)

    value.append(sen[i])
    
    final_vec.append(value)

In [ ]:
test = testing.split()

In [ ]:
wordDicTest = dict.fromkeys(kata_unik, 0)

In [ ]:
for tt in test:
    try:
        wordDicTest[tt] +=1
    except:
        pass

In [ ]:
tfTEst = computeTF(wordDicTest, test)

In [ ]:
idf_test = computeTFIDF(tfTEst, idfs)

In [ ]:
test_vec = []
for k, v in idf_test.items():
    test_vec.append(v)

In [ ]:
def euclidianDistance(sample1, sample2):
    distance = 0
    for x in range(len(sample1)):
        distance += pow(sample1[x] - sample2[x], 2)

    return math.sqrt(distance)

def getNeighbors(train_set, test_sample, k):
    distances = []
    neighbors = []
    
    for x in range(len(train_set)):
        dist = euclidianDistance(test_sample, train_set[x])
        distances.append((train_set[x], dist))
        
    distances.sort(key=operator.itemgetter(1))

    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
            
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [ ]:
def getNeighbors(train_set, test_sample, k):
    distances = []
    neighbors = []
    
    for x in range(len(train_set)):
        dist = euclidianDistance(test_sample, train_set[x])
        distances.append((train_set[x], dist))
        
    distances.sort(key=operator.itemgetter(1))

    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

In [ ]:
def getResponse(neighbors):
    classVotes = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
            
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [ ]:
getResponse(getNeighbors(final_vec, test_vec, k))